# LangChain v1.0 Overview and Migration

LangChain v1.0 doubles down on production agents. The release centers on the new `create_agent`, unified content blocks, and a streamlined namespace while leaning on LangGraph for persistence, streaming, human handoffs, and time travel out of the box.


## What's New in LangChain v1.0

- **`create_agent` as the default**: replaces `langgraph.prebuilt.create_react_agent` with a simpler API and first-class middleware hooks for context engineering and tool orchestration.
- **Middleware pipeline**: built-in PII redaction, summarization, and human approval plus extensible hooks like `before_model`, `wrap_tool_call`, and `after_agent`.
- **Structured outputs in the main loop**: `ToolStrategy` and `ProviderStrategy` eliminate extra model calls and cut cost.
- **Standard content blocks**: provider-agnostic `message.content_blocks` expose reasoning, citations, and tool calls with type hints (Anthropic, AWS, OpenAI, Google, Ollama adapters today).
- **LangGraph benefits included**: checkpoints, streaming, human-in-the-loop, and time travel with zero extra setup.
- **Simplified namespace**: focused exports remain in `langchain.*` while legacy code moves to `langchain-classic`.


### Dummy tools used in the examples

All examples share the same placeholder tools so they can be executed without external dependencies.


In [ ]:
from langchain.tools import tool

DEFAULT_MODEL = "openai:gpt-4o-mini"
PREMIUM_MODEL = "openai:gpt-4o"


@tool
def read_email() -> str:
    """Dummy tool that returns inbox contents."""
    return "No new emails found."


@tool
def send_email(recipient: str, subject: str, body: str) -> str:
    """Dummy tool that pretends to send an email."""
    return f"Email to {recipient} queued."


@tool
def search_web(query: str) -> str:
    """Dummy web search tool."""
    return f"Search results for {query}"


@tool
def analyze_data(dataset: str) -> str:
    """Dummy analytics tool."""
    return f"Analysis for {dataset}"


@tool
def weather_tool(city: str) -> str:
    """Return weather information for a city."""
    return f"It is sunny and 70F in {city}"


assistant_tools = [read_email, send_email, search_web, analyze_data, weather_tool]


## Migration to LangChain v1.0

> Note: install `langchain-classic` and adjust imports if you still rely on legacy chains, indexing, hub, or community exports.


### Agent import

Before:


In [ ]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(
    model=DEFAULT_MODEL,
    tools=assistant_tools,
)

After:


In [ ]:
from langchain.agents import create_agent

agent = create_agent(
    model=DEFAULT_MODEL,
    tools=assistant_tools,
)
agent.invoke(
    {"messages": [{"role": "user", "content": "What is the weather in Berlin?"}]}
)


### System prompt parameter

Before:


In [ ]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(
    model=DEFAULT_MODEL,
    tools=assistant_tools,
    prompt="Be frienly and helpful",
)

After:


In [ ]:
from langchain.agents import create_agent

agent = create_agent(
    model=DEFAULT_MODEL,
    tools=assistant_tools,
    system_prompt="Be friendly and helpful.",
)
agent.invoke(
    {"messages": [{"role": "user", "content": "What is the weather in Berlin?"}]}
)

### Context

In [ ]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(
    model=DEFAULT_MODEL,
    tools=assistant_tools,
)

inputs = {"messages": ["Capital of France?"]}
agent.invoke(inputs, config={"configurable": {"user_id": "42"}})


In [ ]:
from dataclasses import dataclass
from langchain.agents import create_agent


@dataclass
class Context:
    user_id: str


agent = create_agent(
    model=DEFAULT_MODEL,
    tools=assistant_tools,
    context_schema=Context,
)

agent.invoke(inputs, context=Context(user_id="42"))

### Dynamic prompts


In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import dynamic_prompt, ModelRequest

@dataclass
class Context:  
    user_role: str = "user"


@dynamic_prompt
def adjust_prompt(request: ModelRequest) -> str:
    role = request.runtime.context.user_role
    if role == "expert":
        return "You are a deep technical assistant."
    return "Explain concepts in plain language."


agent = create_agent(
    model=DEFAULT_MODEL,
    tools=assistant_tools,
    middleware=[adjust_prompt],
)
agent.invoke(
    {"messages": [{"role": "user", "content": "Explain async programming"}]},
    context=Context(user_role="expert")
)

### Pre-model hook to `before_model`

Before:


In [ ]:
from langgraph.prebuilt import create_react_agent


def summarize_history(state):
    print("Dummy summarize")
    return state


agent = create_react_agent(
    model=DEFAULT_MODEL,
    tools=assistant_tools,
    pre_model_hook=summarize_history,
)


After:


In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware

agent = create_agent(
    model=DEFAULT_MODEL,
    tools=assistant_tools,
    middleware=[
        SummarizationMiddleware(
            model=DEFAULT_MODEL,
            max_tokens_before_summary=500,
        )
    ],
)
agent.invoke(
    {"messages": [...]} # very long list of messages
)

### Post-model hook to `after_model`

Before:


In [ ]:
from langgraph.prebuilt import create_react_agent


def human_review_hook(state, result):
    print("Dummy review")
    return result


agent = create_react_agent(
    model=DEFAULT_MODEL,
    tools=assistant_tools,
    post_model_hook=human_review_hook,
)


After:


In [14]:
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver 

agent = create_agent(
    model=DEFAULT_MODEL,
    tools=assistant_tools,
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "send_email": {"allowed_decisions": ["approve", "reject"]}
            }
        )
    ],
)
agent.invoke(
    {"messages": [{"role": "user", "content": "What is the weather in Berlin?"}]}
)

{'messages': [HumanMessage(content='What is the weather in Berlin?', additional_kwargs={}, response_metadata={}, id='880efa36-3892-4799-9f7d-2203473d1f96'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 135, 'total_tokens': 149, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CUGNcHATF4SmYkB7aEdjoQRbtjnPO', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--2fbbfa93-5956-4fb4-95bb-80e7d2c97733-0', tool_calls=[{'name': 'weather_tool', 'args': {'city': 'Berlin'}, 'id': 'call_4VQjNpoiTHevd3n2XHOGwfba', 'type': 'tool_call'}], usage_metadata={'input_tokens': 135, 'output_tokens': 14, 'to

In [15]:
agent.invoke(
    {"messages": [{"role": "user", "content": "Send an email with 'hi' to my boss"}]}
)

{'messages': [HumanMessage(content="Send an email with 'hi' to my boss", additional_kwargs={}, response_metadata={}, id='19a5c7f1-2a53-4bc9-9ca5-d48556254193'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 138, 'total_tokens': 162, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CUGNpkdOXKcLo6fxJX2dud8VLM7Uc', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--216589c3-7750-43e6-96e1-1dbcacd41777-0', tool_calls=[{'name': 'send_email', 'args': {'recipient': 'boss@example.com', 'subject': 'Hello', 'body': 'hi'}, 'id': 'call_7oYmXPkMYM0XLD7ipQYv08hh', 'type': 'tool_call'}], usage_meta

### Custom state with `AgentState`

Before:


In [16]:
from dataclasses import dataclass


@dataclass
class CustomState:
    user_name: str


After:


In [17]:
from langchain.agents import AgentState


class CustomState(AgentState):
    user_name: str


### Custom Middleware

In [ ]:
from langchain.agents.middleware import AgentMiddleware
from typing_extensions import NotRequired
from typing import Any

class CustomState(AgentState):
    model_call_count: NotRequired[int]


class CallCounterMiddleware(AgentMiddleware[CustomState]):
    state_schema = CustomState  

    def before_model(self, state: CustomState, runtime) -> dict[str, Any] | None:
        count = state.get("model_call_count", 0)
        if count > 10:
            return {"jump_to": "end"}
        return None

    def after_model(self, state: CustomState, runtime) -> dict[str, Any] | None:
        return {"model_call_count": state.get("model_call_count", 0) + 1}


In [ ]:
agent = create_agent(
    model=DEFAULT_MODEL,
    tools=assistant_tools,
    middleware=[CallCounterMiddleware()],
)


### Dynamic model selection


After:


In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import AgentMiddleware, ModelRequest, ModelRequestHandler


class DynamicModel(AgentMiddleware):
    
    def wrap_model_call(self, request: ModelRequest, handler: ModelRequestHandler):
        next_model = PREMIUM_MODEL if len(request.state.messages) > 10 else DEFAULT_MODEL
        return handler(request.replace(model=next_model))


agent = create_agent(
    model=DEFAULT_MODEL,
    tools=assistant_tools,
    middleware=[DynamicModel()],
)


### Structured output with the weather example

After:


In [ ]:
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy
from pydantic import BaseModel


class Weather(BaseModel):
    temperature: float
    condition: str


agent = create_agent(
    model=DEFAULT_MODEL,
    tools=[weather_tool],
    response_format=ToolStrategy(Weather),
)


### Standard content blocks

Before:


In [18]:
from langchain.chat_models import init_chat_model

model = init_chat_model(DEFAULT_MODEL)
response = model.invoke("Capital of France?")
print(response.content)


The capital of France is Paris.


After:


In [19]:
from langchain.chat_models import init_chat_model

model = init_chat_model(DEFAULT_MODEL)
response = model.invoke("Capital of France?")
for block in response.content_blocks:
    if block["type"] == "reasoning":
        print(block["reasoning"])
    elif block["type"] == "text":
        print(block["text"])


The capital of France is Paris.


### Move legacy imports to `langchain-classic`

Before:


In [20]:
try:
    from langchain.chains import LLMChain
    from langchain import hub
except:
    print("Obvious import errors :)")


Obvious import errors :)


After:


In [21]:
from langchain_classic.chains import LLMChain
from langchain_classic import hub
